# Imports

In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch import nn
from helpers import *
from models import *

# Preprocessing

In [2]:
dataset_dir = "./Teeth DataSet/Teeth_Dataset"

In [3]:
train_class_weights, train_class_weights_norm, train_class_counts = show_counts(f"{dataset_dir}/Training")

Class 'CaS' contains 480 images.
Class 'CoS' contains 450 images.
Class 'Gum' contains 360 images.
Class 'MC' contains 540 images.
Class 'OC' contains 324 images.
Class 'OLP' contains 540 images.
Class 'OT' contains 393 images.


In [4]:
train_class_weights_norm

{'CaS': 0.12689335505026225,
 'CoS': 0.13535291205361308,
 'Gum': 0.16919114006701633,
 'MC': 0.11279409337801088,
 'OC': 0.18799015563001817,
 'OLP': 0.11279409337801088,
 'OT': 0.15498425044306838}

In [5]:
im_target_shape = (224, 224)
train_transform = transforms.Compose([
    transforms.Resize(im_target_shape),
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(10),      # Randomly rotate the image by up to 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize(im_target_shape),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = ImageFolder(root=f'{dataset_dir}/Training', transform=train_transform)
val_dataset = ImageFolder(root=f'{dataset_dir}/Validation', transform=test_transform)
test_dataset = ImageFolder(root=f'{dataset_dir}/Testing', transform=test_transform)

batch_size = 16
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


# Model

In [6]:
resnet18 = create_resnet(num_classes=len(train_dataset.classes), resnet_type="resnet18")

c:\Users\001\anaconda3\envs\pytorch-env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\001\anaconda3\envs\pytorch-env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(resnet18.parameters(), lr=0.001, weight_decay=0.0001)
train_class_weights_norm_tensor = torch.tensor(list(train_class_weights_norm.values())).to(device)
criterion = nn.CrossEntropyLoss(weight=train_class_weights_norm_tensor)
num_epochs = 10

In [9]:
train(
    model=resnet18,
    optimizer=optimizer, 
    criterion=criterion, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    num_epochs=num_epochs, 
    device=device,
    models_weights_path="./models_weights/resnet18/best_model.pth")


Epoch 1/10
----------


100%|██████████| 193/193 [00:12<00:00, 15.99it/s]


Train Loss: 1.5031, Accuracy: 43.12%, Precision: 0.4284, Recall: 0.4312, F1 Score: 0.4276


100%|██████████| 65/65 [00:08<00:00,  7.64it/s]


Val Loss: 1.7109, Accuracy: 47.86%, Precision: 0.5907, Recall: 0.4786, F1 Score: 0.4613
--------------------------------------------------
Best model saved with Val Loss: 1.7109
Epoch 2/10
----------


...

Val Loss: 0.2936, Accuracy: 89.88%, Precision: 0.9019, Recall: 0.8988, F1 Score: 0.8988
--------------------------------------------------
Best model saved with Val Loss: 0.2936
Epoch 10/10
----------


100%|██████████| 193/193 [00:10<00:00, 18.03it/s]


Train Loss: 0.3063, Accuracy: 89.37%, Precision: 0.8937, Recall: 0.8937, F1 Score: 0.8935


100%|██████████| 65/65 [00:07<00:00,  9.23it/s]

Val Loss: 0.3377, Accuracy: 87.06%, Precision: 0.8865, Recall: 0.8706, F1 Score: 0.8701
--------------------------------------------------


In [11]:
evaluate(dataloader=test_loader, model=resnet18, device=device, criterion=criterion)

100%|██████████| 65/65 [00:07<00:00,  8.97it/s]


(0.35067928700636214,
 87.64591439688716,
 0.891959017350064,
 0.8764591439688716,
 0.8762765484632277)

In [13]:
train(
    model=resnet18,
    optimizer=optimizer, 
    criterion=criterion, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    num_epochs=20,
    start_epoch=10, 
    device=device,
    models_weights_path="./models_weights/resnet18/best_model.pth")


Epoch 11/20
----------


100%|██████████| 193/193 [00:10<00:00, 18.43it/s]


Train Loss: 0.2612, Accuracy: 90.18%, Precision: 0.9020, Recall: 0.9018, F1 Score: 0.9016


100%|██████████| 65/65 [00:05<00:00, 11.66it/s]


Val Loss: 0.3334, Accuracy: 89.01%, Precision: 0.8969, Recall: 0.8901, F1 Score: 0.8893
--------------------------------------------------
Best model saved with Val Loss: 0.3334
Epoch 12/20
----------


...

Val Loss: 0.1666, Accuracy: 94.16%, Precision: 0.9433, Recall: 0.9416, F1 Score: 0.9418
--------------------------------------------------
Epoch 20/20
----------


100%|██████████| 193/193 [00:11<00:00, 17.52it/s]


Train Loss: 0.1507, Accuracy: 94.59%, Precision: 0.9461, Recall: 0.9459, F1 Score: 0.9458


100%|██████████| 65/65 [00:07<00:00,  8.15it/s]

Val Loss: 0.1400, Accuracy: 95.04%, Precision: 0.9533, Recall: 0.9504, F1 Score: 0.9506
--------------------------------------------------


after 20 epochs

In [18]:
torch.save(resnet18, "./models_weights/resnet18/final_model.pth")

In [14]:
evaluate(dataloader=test_loader, model=resnet18, device=device, criterion=criterion)

100%|██████████| 65/65 [00:07<00:00,  8.53it/s]


(0.17033182693322976,
 95.13618677042801,
 0.9534109405773624,
 0.9513618677042801,
 0.9512956473627326)

In [16]:
m = torch.load("./models_weights/resnet18/best_model.pth")

C:\Users\001\AppData\Local\Temp\ipykernel_15560\3451327770.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  m = torch.load("./models_weights/resnet18/best_model.pth")


best model

In [17]:
evaluate(dataloader=test_loader, model=m, device=device, criterion=criterion)

100%|██████████| 65/65 [00:07<00:00,  9.06it/s]


(0.11528536229821978,
 96.20622568093385,
 0.9626275712457534,
 0.9620622568093385,
 0.9620504900307814)

In [19]:
!pip install streamlit


   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 762.0 kB/s eta 0:00:11
   --- ------------------------------------ 0.8/8.7 MB 860.9 kB/s eta 0:00:10
   --- ------------------------------------ 0.8/8.7 MB 860.9 kB/s eta 0:00:10
   ---- ----------------------------------- 1.0/8.7 MB 898.8 kB/s eta 0:00:09
   ------ --------------------------------- 1.3/8.7 MB 907.1 kB/s eta 0:00:09
   ------- -------------------------------- 1.6/8.7 MB 942.3 kB/s eta 0:00:08
   -------- ------------------------------- 1.8/8.7 MB 967.9 kB/s eta 0:00:08
   --------- ------------------------------ 2.1/8.7 MB 1.0 MB/s eta 0:00:07
   ---------- ----------------------------- 2.4/8.7 MB 994.4 kB/s eta 0:00:07
   ------------ --------------------------- 2.6/8.7 MB 1.0 MB/s eta 0:00:06
   ------------- 

    Markdown (>=3.0<3.3) ; python_version < "3.6"
             ~~~~~~^
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.11 requires pydantic>=2.7.0, but you have pydantic 1.10.17 which is incompatible.
pydantic-extra-types 2.9.0 requires pydantic>=2.5.2, but you have pydantic 1.10.17 which is incompatible.
pydantic-settings 2.4.0 requires pydantic>=2.7.0, but you have pydantic 1.10.17 which is incompatible.


# Streamlit